### Task 3

In [3]:
pip install opencv-python-headless numpy


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.6 MB/s eta 0:00:25
   ---------------------------------------- 0.4/38.8 MB 5.2 MB/s eta 0:00:08
   - -------------------------------------- 1.2/38.8 MB 9.8 MB/s eta 0:00:04
   -- ------------------------------------- 2.1/38.8 MB 12.0 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/38.8 MB 14.2 MB/s eta 0:00:03
   ---- ----------------------------------- 4.2/38.8 MB 15.6 MB/s eta 0:00:03
   ----- ---------------------------------- 5.3/38.8 MB 16.8 MB/s eta 0:00:03
   ------ --------------------------------- 6.4/38.8 MB 17.7 MB/s eta 0:00:02
   ------- -------------------------------- 7.5/38.8 MB 18.4 MB/s eta 0:00:02
   -------- ------------------------------- 8.6/38.8 MB 19.0 MB/s eta 0:00:02
   ---------- ----------------------------- 9.7/38.8 MB 19.5 MB/s eta 0:00:02
   ----------- ---------------------------- 10.8/38.8 MB 23.4 MB/s eta 0:00

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Acer\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



 #### Pre-trained object detection model like YOLOv3 to detect vehicles in the frames.

In [1]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans

def load_yolo(cfg_path, weights_path):
    net = cv2.dnn.readNet(weights_path, cfg_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers


#### Detect Objects

In [2]:
def detect_objects(img, net, output_layers):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    return class_ids, confidences, boxes

In [10]:
def draw_labels(boxes, confidences, class_ids, classes, img):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            if class_ids and i < len(class_ids) and class_ids[i] < len(classes):  # Ensure indices are within bounds
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = (0, 255, 0)
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label, (x, y - 10), font, 1, color, 2)
    return img

def collect_frame_data(frame, class_ids, confidences, boxes):
    frame_data = []
    for class_id, confidence, box in zip(class_ids, confidences, boxes):
        frame_data.append({
            'class_id': class_id,
            'confidence': confidence,
            'box': box
        })
    return frame_data

#### Functions for Traffic Analysis

In [11]:
def calculate_speed(vehicle):
    return np.random.uniform(20, 60)  # Simulating speed calculation

def monitor_traffic_changes(data):
    print(f"Monitoring traffic changes... (Total data points: {len(data)})")

def perform_clustering(data):
    coordinates = []
    for frame_data in data:
        for obj in frame_data:
            box = obj['box']
            coordinates.append([box[0], box[1], box[2], box[3]])
    
    if len(coordinates) >= 3:  # Ensure we have enough data points for clustering
        kmeans = KMeans(n_clusters=3, n_init='auto')
        kmeans.fit(coordinates)
        clusters = kmeans.cluster_centers_
        return clusters
    else:
        return []

def detect_slow_areas(data):
    slow_areas = []
    for frame_data in data:
        for obj in frame_data:
            speed = calculate_speed(obj)
            if speed < 10:  # Assuming a threshold for slow-moving vehicles
                slow_areas.append(obj)
    return slow_areas

def extract_timestamps(data):
    return [np.random.randint(0, 24) for _ in range(len(data))]  # Simulating timestamp extraction

def find_peak_hours(timestamps):
    return np.bincount(timestamps).argmax()  # Simulating peak hour detection

def identify_peak_periods(data):
    return ["08:00-09:00", "17:00-18:00"]  # Simulating peak periods

def detect_congestion_events(data):
    congestion_events = []
    for frame_data in data:
        for obj in frame_data:
            if obj['class_id'] in [0, 1, 2, 3]:  # Assuming class_ids for congestion-causing vehicles
                congestion_events.append(obj)
    return congestion_events

def detect_disruptions(data):
    disruptions = []
    for frame_data in data:
        for obj in frame_data:
            if obj['class_id'] in [0, 1, 2, 3]:  # Assuming class_ids for vehicles causing disruptions
                disruptions.append(obj)
    return disruptions

def optimize_signal_timings(data):
    # Placeholder: Implement logic to recommend signal timings
    return {"08:00-09:00": 30, "17:00-18:00": 45}  # Simulating signal timings

def coordinate_signals(data):
    # Placeholder: Implement logic to coordinate signals
    return ["Intersection 1 and 2", "Intersection 3 and 4"]  # Simulating signal coordination

def analyze_traffic_flow(data):
    speeds = [calculate_speed(vehicle) for vehicle in data]
    average_speed = np.mean(speeds) if speeds else 0
    print(f"Average speed: {average_speed} km/h")
    
    num_vehicles = len(data)
    area_of_interest = 1  # Placeholder for the actual area
    density = num_vehicles / area_of_interest if area_of_interest else 0
    print(f"Traffic density: {density} vehicles/square km")
    
    monitor_traffic_changes(data)

def identify_congestion_hotspots(data):
    clusters = perform_clustering(data)
    print(f"Detected {len(clusters)} congestion hotspots.")
    
    slow_areas = detect_slow_areas(data)
    for area in slow_areas:
        print(f"Slow-moving area detected at {area['box']}.")

def detect_peak_traffic_times(data):
    timestamps = extract_timestamps(data)
    peak_hours = find_peak_hours(timestamps)
    print(f"Peak traffic times: {peak_hours}")
    
    peak_periods = identify_peak_periods(data)
    print(f"Peak traffic periods detected: {peak_periods}")

def analyze_congestion_causes(data):
    congestion_events = detect_congestion_events(data)
    for event in congestion_events:
        print(f"Congestion event detected: {event}")
    
    disruptions = detect_disruptions(data)
    for disruption in disruptions:
        print(f"Traffic disruption detected: {disruption}")

def recommend_signal_optimizations(data):
    optimized_timings = optimize_signal_timings(data)
    print(f"Recommended signal timings: {optimized_timings}")
    
    coordinated_signals = coordinate_signals(data)
    print(f"Signal coordination recommended: {coordinated_signals}")


In [12]:
def process_video(video_path, output_folder, net, output_layers, classes):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        class_ids, confidences, boxes = detect_objects(frame, net, output_layers)
        frame = draw_labels(boxes, confidences, class_ids, classes, frame)
        
        frame_data = collect_frame_data(frame, class_ids, confidences, boxes)
        data.append(frame_data)
        
        output_path = os.path.join(output_folder, f'frame_{frame_id:04d}.jpg')
        cv2.imwrite(output_path, frame)
        print(f'Processed frame {frame_id} and saved to {output_path}')
        frame_id += 1

    cap.release()
    
    analyze_traffic_flow(data)
    identify_congestion_hotspots(data)
    detect_peak_traffic_times(data)
    analyze_congestion_causes(data)
    recommend_signal_optimizations(data)

#### USECASE Example

In [13]:
if __name__ == '__main__':
    cfg_path = 'yolov3.cfg'
    weights_path = 'yolov3.weights'
    video_path = 'traffic/test01.avi'
    output_folder = 'processed_frames'
    classes = ["car", "motorbike", "bus", "truck"]

    net, output_layers = load_yolo(cfg_path, weights_path)
    print("YOLOv3 model loaded successfully!")
    process_video(video_path, output_folder, net, output_layers, classes)

YOLOv3 model loaded successfully!
Processed frame 0 and saved to processed_frames\frame_0000.jpg
Processed frame 1 and saved to processed_frames\frame_0001.jpg
Processed frame 2 and saved to processed_frames\frame_0002.jpg
Processed frame 3 and saved to processed_frames\frame_0003.jpg
Processed frame 4 and saved to processed_frames\frame_0004.jpg
Processed frame 5 and saved to processed_frames\frame_0005.jpg
Processed frame 6 and saved to processed_frames\frame_0006.jpg
Processed frame 7 and saved to processed_frames\frame_0007.jpg
Processed frame 8 and saved to processed_frames\frame_0008.jpg
Processed frame 9 and saved to processed_frames\frame_0009.jpg
Processed frame 10 and saved to processed_frames\frame_0010.jpg
Processed frame 11 and saved to processed_frames\frame_0011.jpg
Processed frame 12 and saved to processed_frames\frame_0012.jpg
Processed frame 13 and saved to processed_frames\frame_0013.jpg
Processed frame 14 and saved to processed_frames\frame_0014.jpg
Processed frame 

C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


### Recommendations for Traffic Signal Optimization

1. **Adaptive Signal Control**:
   Implement adaptive signal control systems that adjust signal timings based on real-time traffic conditions. This can help manage traffic flow more effectively and reduce congestion during peak hours.

2. **Traffic Signal Synchronization**:
   Synchronize traffic signals along major corridors to ensure smooth traffic flow. This can reduce stop-and-go driving, minimize delays, and improve fuel efficiency.

3. **Peak Hour Management**:
   Introduce measures to manage peak hour traffic, such as dedicated lanes for high-occupancy vehicles (HOV) and encouraging alternative transportation methods during peak times.

4. **Congestion Monitoring**:
   Continuously monitor congestion hotspots and adjust traffic signal timings accordingly. This will help in dynamically managing traffic flow and preventing bottlenecks.

### Communication of Findings

- Present the analysis and recommendations to city officials through comprehensive reports and visualizations.
- Use dashboards and real-time monitoring tools to keep officials informed about traffic conditions and the impact of implemented strategies.
